Add Imports

In [2]:
import asyncio
from functools import wraps
from langchain.llms import openai
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.document_loaders import CSVLoader
from langchain.chains.summarize import load_summarize_chain
import csv
from langchain.docstore.document import Document
from typing import List
from langchain.prompts import PromptTemplate
from pathlib import Path
from langchain.document_loaders.csv_loader import CSVLoader
from openai import OpenAIError
import tiktoken
from langchain.text_splitter import CharacterTextSplitter
import csv
from collections import namedtuple

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate)


Load CSV In / to be from db and chunked later.

In [13]:
import pandas as pd

# Load the data
df = pd.read_csv(r'C:\Own Your AI GIT\SiteAI-Backend\app\logic\menu\cactus-min.csv')

# Replace all None/NaN values with an empty string
df.fillna('', inplace=True)

# Save the data again
df.to_csv(r'C:\Own Your AI GIT\SiteAI-Backend\app\logic\menu\cactus-min.csv', index=False)

loader = CSVLoader(file_path=r'C:\Own Your AI GIT\SiteAI-Backend\app\logic\menu\cactus-min.csv', source_column="Unique ID", csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['Unique ID', 'Sub Category', 'Name', 'Description', 'Price', 'Upgrades', 'Tags', 'SERVING SIZE (G)', 'CALORIES (KCAL)', 'TOTAL FAT (G)', 'SATURATED FAT (G)', 'TRANSFAT (G)', 'CHOLESTEROL (MG)', 'SODIUM (MG)', 'CARBOHYDRATE (G)', 'DIETARY FIBRE (G)', 'SUGARS (G)', 'PROTEIN (G)']
})

documents = loader.load()

# Clean up the newline characters in each document's page_content
for document in documents:
    document.page_content = document.page_content.replace("\n", " ")  # Replace newline characters with a space
    
data = loader.load()


AttributeError: 'NoneType' object has no attribute 'strip'

In [7]:
print(f"Collected  {len(data)} Rows")
# print (data[10].page_content)
print (data)


NameError: name 'data' is not defined

## CURRENT CODE BREAKS DOWN MENU ITEM AT CUTS I BELIEVE

In [9]:

loader = CSVLoader(file_path=r'"C:\Users\BDru\Downloads\cactus-min.csv"', source_column="Unique ID", csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['Unique ID', 'Sub Category', 'Name', 'Description', 'Price', 'Upgrades', 'Tags', 'SERVING SIZE (G)', 'CALORIES (KCAL)', 'TOTAL FAT (G)', 'SATURATED FAT (G)', 'TRANSFAT (G)', 'CHOLESTEROL (MG)', 'SODIUM (MG)', 'CARBOHYDRATE (G)', 'DIETARY FIBRE (G)', 'SUGARS (G)', 'PROTEIN (G)']
})


documents = loader.load()

# Clean up the newline characters in each document's page_content
for document in documents:
    document.page_content = document.page_content.replace("\n", " ")  # Replace newline characters with a space
    

data = loader.load()


encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def count_tokens(text):
    return len(encoding.encode(text))

def group_documents(data, max_chunk_tokens=1700):
    # Group chunks into groups
    groups = []
    group = []
    current_tokens = 0
    for chunk_list in data:
        for doc in chunk_list:
            doc_tokens = count_tokens(doc)
            # Check if adding this chunk will exceed the max_chunk_tokens limit
            if current_tokens + doc_tokens > max_chunk_tokens:
                # If so, start a new group
                groups.append(group)
                group = [doc]
                current_tokens = doc_tokens
            else:
                # If not, add the chunk to the current group
                group.append(doc)
                current_tokens += doc_tokens

    # Add the last group if it isn't empty
    if group:
        groups.append(group)

    return groups


def clean_page_content(page_content):
    lines = page_content.split("\n")
    clean_lines = [line for line in lines if line.strip()]  # Removes empty lines
    return ' '.join(clean_lines)


def custom_split_text(page_content, max_chunk_tokens):  # Renamed max_chunk_size to max_chunk_tokens for clarity
    text = clean_page_content(page_content).split('.')  # Split text into sentences
    chunks = []
    current_chunk = ""

    for index, sentence in enumerate(text):  # Iterate over sentences
        # Ignore empty sentences
        if not sentence.strip():
            continue
            
        sentence_tokens = count_tokens(sentence)
        current_chunk_tokens = count_tokens(current_chunk)
        
        # If current chunk plus new sentence exceeds chunk size or it is the last sentence
        if current_chunk_tokens + sentence_tokens > max_chunk_tokens or index == len(text) - 1:
            # If the current chunk isn't empty, save it
            if current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = ""
            
            # If the sentence itself is larger than chunk size, split it into multiple chunks
            if sentence_tokens > max_chunk_tokens:
                for i in range(0, len(sentence), max_chunk_tokens):
                    chunks.append(sentence[i:i+max_chunk_tokens])
            else:
                # Add the remaining part of the sentence to current chunk
                current_chunk += sentence + " "
                
        # If current chunk plus new sentence doesn't exceed chunk size, add sentence to current chunk
        else:
            current_chunk += sentence + " "

    # Save the last chunk if it's not empty
    if current_chunk.strip():
        chunks.append(current_chunk.strip())

    return chunks

# Here, create_chunked_docs is equivalent to custom_split_text
create_chunked_docs = custom_split_text

chunks = [create_chunked_docs(doc.page_content, max_chunk_tokens=3000) for doc in data]

# Now group these chunks
grouped_chunks = group_documents(chunks, max_chunk_tokens=3000)


OSError: [Errno 22] Invalid argument: '"C:\\Users\\BDru\\Downloads\\cactus-min.csv"'

In [2]:
print(f"Grouped into {len(grouped_chunks)} grouped chunks")
#add random note
# print (grouped_chunks[2])

NameError: name 'grouped_chunks' is not defined

In [ ]:
'''* add history to the chatbot 
History
System - link to front end (Filter system prompt 1)
User - Prmopt
System - link to front end (Filter system prompt 2)

**
'''
from tenacity import retry, stop_after_attempt, wait_fixed
import threading
nest_asyncio.apply()



human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="""
            
            As a representative of Cactus Club Cafe, you are tasked with delivering an exceptional customer service experience by being attentive to the customers' needs. 
            
            Your job is to review the customers chat message, and our menu, and pick up to 5 menu items that best match the customers' query, and output them in rank order.
            Output them always in the following format: Unique IDs only, space seperated, max 5 results.

            Here are some examples of customer queries:

            Request: I only eat plant-based foods.
            Strategy: Check for menu sections labeled "Vegetarian" or "Vegan," or filter items by key ingredients like tofu, seitan, and vegetables.

            Request: I'm allergic to peanuts.
            Strategy: Avoid items explicitly listed with peanuts or peanut sauce, and opt for dishes labeled as 'nut-free' or ask for modifications if possible.

            Request: I'm trying to eat less sugar.
            Strategy: Avoid dishes with descriptions that include words like "sweet," "glazed," or "caramelized" and look for meals high in proteins and vegetables instead.

            Request: I need gluten-free options.
            Strategy: Look for dishes specifically labeled as 'gluten-free', or filter for meals that primarily consist of meats, vegetables, and naturally gluten-free grains like rice or quinoa.

            Request: I'm looking for high-protein meals.
            Strategy: Focus on menu items that feature lean meats, fish, tofu, or legumes as the main ingredient.

            Request: I have to watch my sodium intake.
            Strategy: Stay away from dishes that are described as "salty," "brined," "pickled," or "smoked," and aim for fresh ingredients and lighter sauces.

            Request: I'm avoiding dairy.
            Strategy: Opt for meals that don't mention cheese, milk, cream, or butter in their description, and search for 'dairy-free' or 'vegan' options if available.

            Request: I'm on a ketogenic diet.
            Strategy: Look for items high in healthy fats and proteins and low in carbs, typically meat, fish, and non-starchy vegetable dishes.

            Request: I'm looking for a high-fiber meal.
            Strategy: Target dishes that include whole grains, legumes, fruits, and vegetables as primary ingredients.

            Request: I'm avoiding red meat.
            Strategy: Scan the menu for dishes featuring poultry, fish, tofu, or plant-based proteins as main ingredients.

            Request: I need a low-carb meal.
            Strategy: Look for dishes that prioritize proteins and non-starchy vegetables and avoid those with pasta, bread, or rice.

            Request: I want to avoid spicy food.
            Strategy: Avoid dishes marked with peppers or that mention words like "spicy," "hot," "szechuan," or "curry" in their descriptions.

            Request: I'm looking for organic meals.
            Strategy: Look for the 'organic' label in the dish description or identify menu sections specifically dedicated to organic options.

            Request: I'm avoiding seafood.
            Strategy: Bypass dishes that include fish, shellfish, or any other form of seafood in the description, and focus on meat, poultry, or plant-based dishes.

            Request: I need meals with low fat content.
            Strategy: Opt for dishes that feature lean meats, whole grains, and vegetables, and avoid items described as "fried," "battered," or "creamy."

            Our Menu: {menu} \n\n Provide No more than 5 matches. If no matches, output 'null'. Rank order matches.  Output format, Unique IDs only, space seperated, max 5 results : ",

            
            Customer Query: {query} \n\n 

            Results (Unique IDs only, space seperated, max 5 results)): 
            """,
            input_variables=["query", "menu"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.0)
chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=False)
# This decorator will make the function retry up to 5 times, waiting 2 seconds between attempts
@retry(stop=stop_after_attempt(5), wait=wait_fixed(0.1))
def run_chat(chain, query, menu, results):
    result = chain.run({
        'query': query,
        'menu': menu
    })
    results.append(result)

def run_chats_with_retry(chain, query, grouped_chunks):
    results = []

    threads = [threading.Thread(target=run_chat, args=(chain, query, chunk, results)) for chunk in grouped_chunks]

    # Start all threads
    for thread in threads:
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    # Return the results
    return results
'''
    Submenus
    1. Appetizers
    2. Salads
    3. Soups
    4. Sandwiches    
    5. Burgers
    [Appetizers, Salads, Soups, Sandwiches, Burgers]
    main_tool (query) -> decided up to 3 submenus to choose from
    tool_sub_menu_a -> 
    tool_sub_menu_b -> 
    tool_sub_menu_c ->
    
'''


query = " I need something kosher."
grouped_chunks = grouped_chunks
results = run_chats_with_retry(chain, query, grouped_chunks)


In [ ]:
print(results)

['null', 'null', 'null', 'null', 'null']


In [ ]:
def match_unique_ids(output, data, max_results):
    menu_chat = []
    found_ids = []
    missing_ids = []
    
    # Transform output into a list of lists
    output = [group.split() for group in output]

    # Continue until we have enough results, or all IDs are used
    while len(menu_chat) < max_results and any(output):
        # Loop through each group
        for group in output:
            # If the group is not empty
            if group:
                # Get the first ID in the group
                id = group.pop(0)
                # Search for the ID in data
                found = False
                for item in data:
                    if 'Unique ID: ' + id in item.page_content:
                        menu_chat.append(item)
                        found_ids.append(id)
                        found = True
                        break
                if not found:
                    missing_ids.append(id)
                # Stop if we have enough results
                if len(menu_chat) == max_results:
                    break

    # Print IDs not found in data
    if missing_ids:
        print('IDs not found in data:', ', '.join(missing_ids))

    return menu_chat




# Your `data` list for matching
# data = ['Unique ID: A061\nSub Category: GLUTEN SMART\nName: CALAMARI BRAVAS\nDescription: ten....IN (G): 36', 'Unique ID: A062...']

# maximum results
max_results = 10

# Call the function with your results, your data, and max results
menu_chat = match_unique_ids(results, data, max_results)



IDs not found in data: null, null, null, null, null


In [ ]:
print(menu_chat)

[]


In [ ]:
'''* add history to the chatbot 
History
System - link to front end (Response system prompt 1)
User - Prmopt
System - link to front end (Response system prompt 2)

**
'''


from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

template = """  
            You are Cactus Club Cafe, Read the customer's query and chat history carefully and identify customer needs, then review autogenerated catalog and use relevant results to align with an optimal response to the client. 

            Please follow these guidelines:


                1. Base your answer on the customer's query and chat history.
                2. Use the manu to provide recomendations and discuss food options.
                3. When discussing our menu options, use the format: [#A001]Title[/#A001], where the number is the menu Unique ID reference number. This format is required by our system.
                4. Always Be: concise, professional, friendly, and non confronatational.
                5. When appropriate, be humorous, polite, and respectful. 
                6. Do not engage in any controversial topics or lude comments.
                7. Respond in the customer's language
                8. End conversations with a proactive question about enhancing their meal (e.g., "Would you like to find a few great wine or cocktail options to complement your meal?").
                9. Do not mention other restaurants or web resources. 
                10. If you do not know an answer, apologies and direct them to confirm with a member of our service team in the restaurant.
                11. Do not mention the catalog to the customer.
                12. We do substitions and customizations, where needed direc them to our service team in the restaurant and they will do their best to satisfy the customers request.


                If the customer's query is unclear, ask clarifying questions to better understand their needs.

                Example:
                Query: I'm looking for a delicious vegetarian dish for dinner tonight.
                Response: We have some delightful vegetarian options that you'll surely love! For starters, consider the 🥗 [#E005]Quinoa and Avocado Powerbowl[/#E005]. This dish is brimming with nutritious ingredients such as quinoa, avocado, cucumber, and cherry tomatoes, all drizzled with a zesty tahini dressing. Alternatively, you might enjoy the [#E008]Pesto and Goat Cheese Flatbread[/#E0088]. This gourmet flatbread features creamy goat cheese, vibrant basil pesto, and a medley of fresh veggies. Both are great choices! While I did my best to give you only vegetarian options Can I assist you with anything else, perhaps a drink to complement your meal?

               


                Menu Extract: {menu_chat}
 """

# Define the variables

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{query}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Format the prompt using the variables
formatted_messages = chat_prompt.format_prompt(query=query, menu_chat=menu_chat).to_messages()

# Create a ChatOpenAI instance
chat = ChatOpenAI(temperature=0)

# Send the formatted messages to the chat model
response = chat(formatted_messages)

# Print the AI-generated message content

print(response.content)

Absolutely! We have a few kosher options on our menu that I think you'll enjoy. For a delicious appetizer, I recommend the 🍤 [#A003]Jumbo Prawn Cocktail[/#A003]. For your main course, we have the 🥩 [#A005]Certified Angus Beef® Prime Rib[/#A005] which is served with a side of horseradish and au jus. Another great option is the 🐟 [#A007]Wild Sockeye Salmon[/#A007] which is served with a side of seasonal vegetables. All of these dishes are kosher and are sure to satisfy your cravings. Is there anything else I can help you with?


In [ ]:
print(formatted_messages)

[SystemMessage(content='  As a representative of Cactus Club Cafe, your role is to provide excellent customer service by understanding and meeting the customer\'s needs. This includes carefully reading the customer\'s query and chat history, and then reviewing our catalog to find the best dishes that align with their preferences.\n\n                When referencing the catalog, use the format: [#A001]Title[/#A001], where the number is the menu Unique ID reference number. This format is required by our system.\n\n                Your response should be conversational, highlighting the benefits of each dish in relation to the customer\'s needs. Offer options that pair well with the selected dishes to enhance the dining experience. When providing multiple options, contrast variants of the same category, highlight the advantages of each, and how they relate to the customer\'s needs.\n\n                Maintain a tone that is concise, professional, friendly, humorous, polite, and respectful

In [ ]:
        connection = mysql.connector.connect(
            host=os.getenv('CLIENT_DB_HOST'),
            user=os.getenv('CLIENT_DB_USER'),
            password=os.getenv('CLIENT_DB_PASSWORD'),
            database=os.getenv('CLIENT_DB_DATABASE'),
            port=os.getenv('CLIENT_DB_PORT')
        )
        if connection.is_connected():
            return connection
    except Error as e:
        print("Error while connecting to MySQL", e)
        return None, None